In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Cafe_II").getOrCreate()
cafe_df = spark.read.json("filmaking.json")
cafe_df.createOrReplaceTempView("filmaking")

cafe_df.show(30)
cafe_df.printSchema()
       

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/23 19:33:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/23 19:33:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
                                                                                

+------------+---------------+---+------------------+---------------+---------+--------------------+
|duracion_min|    herramienta| id|            premio|presupuesto_usd|      rol|              titulo|
+------------+---------------+---+------------------+---------------+---------+--------------------+
|          15|  Final Cut Pro|  1|Mejor Cortometraje|           1200| director|     Luz de Invierno|
|           8|DaVinci Resolve|  2|              NULL|            500|   editor|      Café y Sombras|
|          12|DaVinci Resolve|  3|              NULL|            800|colorista|     El Último Pixel|
|          20|    Final Draft|  4|   Guion Destacado|           1500|guionista|     Sueños de Acero|
|          10|  Final Cut Pro|  5|              NULL|            900| director|     Ráfaga Nocturna|
|           7| Adobe Premiere|  6|              NULL|            400|   editor|          Fragmentos|
|          18|  Final Cut Pro|  7| Selección Oficial|           2000| director|      Vértig

In [5]:
# 1. ¿Cuántos proyectos tienen una duración mayor al promedio de todos los proyectos?
spark.sql(""" 
            SELECT COUNT(*)  
            FROM filmaking 
            WHERE duracion_min > (SELECT AVG(duracion_min) FROM filmaking);
""").show()

+--------+
|count(1)|
+--------+
|      15|
+--------+



In [13]:
# 2. Lista los títulos de los proyectos que han ganado premio y 
# cuyo presupuesto es menor al presupuesto promedio de todos los proyectos premiados.
spark.sql(""" 
            SELECT titulo, premio
            FROM filmaking
            WHERE presupuesto_usd < (SELECT AVG(presupuesto_usd) FROM filmaking WHERE premio IS NOT NULL) AND premio IS NOT NULL
            ORDER BY titulo ASC;
""").show()

+------------------+------------------+
|            titulo|            premio|
+------------------+------------------+
|Guion de Esperanza|Mención Honorífica|
|   Luz de Invierno|Mejor Cortometraje|
|   Sueños de Acero|   Guion Destacado|
+------------------+------------------+



In [17]:
# ¿Cuál es el rol que más veces aparece en la base de datos?
spark.sql(""" 
            SELECT rol, COUNT(*) AS veces_que_se_repite
            FROM filmaking
            GROUP BY rol
            ORDER BY rol DESC
            LIMIT 1;
""").show()

+---------+-------------------+
|      rol|veces_que_se_repite|
+---------+-------------------+
|guionista|                  8|
+---------+-------------------+



In [20]:
# 4. Muestra el título y la herramienta de los proyectos cuyo título contiene la palabra "Luz" (sin importar mayúsculas/minúsculas).
spark.sql(""" 
            SELECT titulo, herramienta
            FROM filmaking
            WHERE LOWER(titulo) LIKE "%luz%";
""").show()

+---------------+-------------+
|         titulo|  herramienta|
+---------------+-------------+
|Luz de Invierno|Final Cut Pro|
|    Ritmo y Luz|Final Cut Pro|
| Caminos de Luz|Final Cut Pro|
+---------------+-------------+



In [21]:
# 5. ¿Cuántos proyectos tienen un presupuesto entre $500 y $1500 USD (incluyendo ambos extremos)?
spark.sql(""" 
            SELECT COUNT(*) AS proyectos_entre_500_y_1500_inclusive
            FROM filmaking
            WHERE presupuesto_usd BETWEEN 500 AND 1500          
""").show()

+------------------------------------+
|proyectos_entre_500_y_1500_inclusive|
+------------------------------------+
|                                  19|
+------------------------------------+



In [22]:
# 6. ¿Cuál es el promedio de duración de los proyectos que no han ganado ningún premio?
spark.sql(""" 
            SELECT AVG(duracion_min) AS promedio_duracion
            FROM filmaking
            WHERE premio IS NULL;
""").show()

+------------------+
| promedio_duracion|
+------------------+
|11.347826086956522|
+------------------+



In [24]:
# 7. Lista los títulos de los proyectos cuyo rol es “guionista” y que tienen una duración mayor a 15 minutos.
spark.sql(""" 
            SELECT titulo
            FROM filmaking
            WHERE rol = "guionista" and duracion_min > 15;
""").show()

+------------------+
|            titulo|
+------------------+
|   Sueños de Acero|
|Horizonte de Papel|
| Voces del Asfalto|
+------------------+



In [30]:
# 8. ¿Cuántos proyectos usan “Motion 5” y tienen una duración menor o igual a 6 minutos?
spark.sql(""" 
            SELECT COUNT (*) AS Motion_5
            FROM filmaking
            WHERE herramienta = "Motion 5" and duracion_min <= 6;
""").show()

+--------+
|Motion_5|
+--------+
|       3|
+--------+



In [28]:
# 9. Muestra el título y presupuesto de los tres proyectos más caros.
spark.sql(""" 
            SELECT titulo, presupuesto_usd
            FROM filmaking
            ORDER BY presupuesto_usd DESC
            LIMIT 3;
""").show()

+------------------+---------------+
|            titulo|presupuesto_usd|
+------------------+---------------+
|  Caminos Cruzados|           2500|
|Laberinto de Ideas|           2100|
|    Vértigo Urbano|           2000|
+------------------+---------------+



In [33]:
# 10. ¿Cuántos proyectos tienen la palabra “Guion” en el título y han ganado algún premio?
spark.sql(""" 
            SELECT COUNT(*) AS palabra_guion_inside
            FROM filmaking
            WHERE LOWER (titulo) LIKE "%guion%" and premio IS NOT NULL;
""").show()

+--------------------+
|palabra_guion_inside|
+--------------------+
|                   1|
+--------------------+



In [40]:
spark.sql(""" 
SELECT rol, COUNT(*) as total_proyectos
FROM filmaking
GROUP BY rol
HAVING COUNT(*) > 3;
""").show()

+---------+---------------+
|      rol|total_proyectos|
+---------+---------------+
| director|              9|
|   editor|              9|
|guionista|              8|
+---------+---------------+

